In [0]:
import random
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
import itertools

In [0]:
# import random, string
# password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

# #Download ngrok
# ! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# ! unzip -qq -n ngrok-stable-linux-amd64.zip
# #Setup sshd
# ! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
# #Set root password
# ! echo root:$password | chpasswd
# ! mkdir -p /var/run/sshd
# ! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
# ! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
# ! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
# ! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

# #Run sshd
# get_ipython().system_raw('/usr/sbin/sshd -D &')

# #Ask token
# print("Copy authtoken from https://dashboard.ngrok.com/auth")
# import getpass
# authtoken = getpass.getpass()

# #Create tunnel
# get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')
# #Print root password
# print("Root password: {}".format(password))
# #Get public address
# ! curl -s http://localhost:4040/api/tunnels | python3 -c \
#     "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [205]:
!nvidia-smi

Thu Mar 12 13:11:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# ! curl -s http://localhost:4040/api/tunnels | python3 -c \
#     "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
#!wget https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/download/P5ocxmvCguW59E8YFZnh%2Fversions%2FUNHbff4Y6pynSWzk4vQQ%2Ffiles%2Ftest.tsv.zip

In [3]:
!unzip train.tsv.zip
!unzip test.tsv.zip

Archive:  train.tsv.zip
  inflating: train.tsv               
Archive:  test.tsv.zip
  inflating: test.tsv                


In [0]:
train_df = pd.read_csv('train.tsv', delimiter='\t')

In [207]:
train_df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [0]:
params = {'C': [1.0, 0.1], 'fit_intercept':[False], 'ngram_range':[(1,3)], 'max_features':[10000, 5000], 'max_iter':[1000, 500], 'use_tf_idf':[True, False], 'analyzer':['word']}

In [0]:
class TrainAgent():
    def __init__(self, combination):
        self.combination = combination

    def get_performance(self, X_train, X_test, y_train, y_test):
        combination = self.combination

        C = combination['C']
        fit_intercept=combination['fit_intercept']
        max_features = combination['max_features']
        ngram_range = combination['ngram_range']
        max_iter = combination['max_iter']
        use_tf_idf = combination['use_tf_idf']
        analyzer = combination['analyzer']

        if use_tf_idf:
            vect = TfidfVectorizer(max_features=max_features, ngram_range=ngram_range, analyzer=analyzer, dtype=np.float32)
        else:
            vect = CountVectorizer(max_features=max_features, ngram_range=ngram_range, analyzer=analyzer, dtype=np.float32)

        vect = vect.fit(X_train)
        X_train_i = vect.transform(X_train)
        X_test_i = vect.transform(X_test)
        clf = LogisticRegression(C=C, fit_intercept=fit_intercept, max_iter=max_iter)
        clf.fit(X_train_i, y_train)
        y_predictions = clf.predict(X_test_i)
        return accuracy_score(y_test, y_predictions)


class Optimus():
    def __init__(self, params):
        self._recalculate_combinations(params)

    def _recalculate_combinations(self, params):
        self.params = params
        self.combinations = self._get_all_combinations(self.params)
        print('Optimus hyper-space is instantiated')
        print(f'Total: {len(self.combinations)} possible combinations:')
        print(self.combinations)


    def _find_prospective_difference(self, dict_1, dict_2):
        accuracy_1 = dict_1[0]
        accuracy_2 = dict_2[0]
        ls_1 = dict_1[1]
        ls_2 = dict_2[1]
        keys_1 = ls_1.keys()
        keys_2 = ls_2.keys()
        int_keys = ['max_iter', 'max_features']
        float_keys = ['C']
        prospective_params = {k:[] for k in keys_1}
        for (i, (key_1, key_2)) in enumerate(zip(keys_1, keys_2)):
            value_1 = ls_1[key_1]
            value_2 = ls_2[key_2]
            if value_1 >= value_2 and accuracy_1 >= accuracy_2 and key_1 in float_keys:
                prospective_params[key_1].append(value_1 + 0.1)
            elif value_1 <= value_2 and accuracy_1 >= accuracy_2 and key_1 in float_keys:
                prospective_params[key_1].append(value_1 - 0.1)
            elif value_1 >= value_2 and accuracy_1 >= accuracy_2 and key_1 in int_keys:
                prospective_params[key_1].append(value_1 + 5)
            elif value_1 <= value_2 and accuracy_1 >= accuracy_2 and key_1 in int_keys:
                prospective_params[key_1].append(value_1 - 5)
        return prospective_params

    def _adjust_params_from_agents(self, iteration_result):
        best_2_agents = iteration_result[0:2]
        result = self.params
        for i in best_2_agents:
            for j in best_2_agents:
                prospective_params = self._find_prospective_difference(i, j)
                for key in self.params.keys():
                    values = prospective_params[key]
                    for v in values:
                        if v not in self.params[key]:
                            self.params[key].append(v)
                            self.params[key] = self.params[key][1:]
        return self.params

    def _get_all_combinations(self, params):
        return [{'C' : x,'fit_intercept' : y, 
                 'ngram_range' : z, 
                 'max_features' : i, 
                 'max_iter': j, 
                 'use_tf_idf':k, 
                 'analyzer':l} 
                for x in params['C'] 
                for y in params['fit_intercept'] 
                for z in params['ngram_range']
                for i in params['max_features']
                for j in params['max_iter']
                for k in params['use_tf_idf']
                for l in params['analyzer']]

    def _perform_hyper_search(self, train):
        X_train = train['Phrase'].values
        y_train = train['Sentiment'].values
        X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
        result = []
        for combination in self.combinations:
            agent = TrainAgent(combination)
            accuracy = agent.get_performance(X_train, X_test, y_train, y_test)
            result.append((accuracy, combination))
            print(f'Agent is spanned with current combination: {combination}')
            print(f'Accuracy for combination is: {accuracy}')
        return sorted(result, key=lambda x: x[0], reverse=True)

    def perform_optimus_search(self, train, iter = 3):
        for i in range(iter):
            print(f'Iteration: {i}')
            iteration_result = self._perform_hyper_search(train)
            new_params = self._adjust_params_from_agents(iteration_result)
            self._recalculate_combinations(new_params)
        return iteration_result

In [210]:
optimus = Optimus(params)

Optimus hyper-space is instantiated
Total: 16 possible combinations:
[{'C': 1.0, 'fit_intercept': False, 'ngram_range': (1, 3), 'max_features': 10000, 'max_iter': 1000, 'use_tf_idf': True, 'analyzer': 'word'}, {'C': 1.0, 'fit_intercept': False, 'ngram_range': (1, 3), 'max_features': 10000, 'max_iter': 1000, 'use_tf_idf': False, 'analyzer': 'word'}, {'C': 1.0, 'fit_intercept': False, 'ngram_range': (1, 3), 'max_features': 10000, 'max_iter': 500, 'use_tf_idf': True, 'analyzer': 'word'}, {'C': 1.0, 'fit_intercept': False, 'ngram_range': (1, 3), 'max_features': 10000, 'max_iter': 500, 'use_tf_idf': False, 'analyzer': 'word'}, {'C': 1.0, 'fit_intercept': False, 'ngram_range': (1, 3), 'max_features': 5000, 'max_iter': 1000, 'use_tf_idf': True, 'analyzer': 'word'}, {'C': 1.0, 'fit_intercept': False, 'ngram_range': (1, 3), 'max_features': 5000, 'max_iter': 1000, 'use_tf_idf': False, 'analyzer': 'word'}, {'C': 1.0, 'fit_intercept': False, 'ngram_range': (1, 3), 'max_features': 5000, 'max_iter':

In [211]:
final_result = optimus.perform_optimus_search(train_df)

Iteration: 0
Agent is spanned with current combination: {'C': 1.0, 'fit_intercept': False, 'ngram_range': (1, 3), 'max_features': 10000, 'max_iter': 1000, 'use_tf_idf': True, 'analyzer': 'word'}
Accuracy for combination is: 0.525599128540305
Agent is spanned with current combination: {'C': 1.0, 'fit_intercept': False, 'ngram_range': (1, 3), 'max_features': 10000, 'max_iter': 1000, 'use_tf_idf': False, 'analyzer': 'word'}
Accuracy for combination is: 0.5087145969498911
Agent is spanned with current combination: {'C': 1.0, 'fit_intercept': False, 'ngram_range': (1, 3), 'max_features': 10000, 'max_iter': 500, 'use_tf_idf': True, 'analyzer': 'word'}
Accuracy for combination is: 0.525599128540305
Agent is spanned with current combination: {'C': 1.0, 'fit_intercept': False, 'ngram_range': (1, 3), 'max_features': 10000, 'max_iter': 500, 'use_tf_idf': False, 'analyzer': 'word'}
Accuracy for combination is: 0.5087145969498911
Agent is spanned with current combination: {'C': 1.0, 'fit_intercept'